In [18]:
import math


In [19]:
class DecisionStump:
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.left_value = None
        self.right_value = None
    def fit(self, X, y):
        n_samples = len(X)
        n_features = len(X[0])
        best_error = float("inf")

        for feature in range(n_features):
            thresholds = sorted(set(row[feature] for row in X))

            for threshold in thresholds:
                left_y = []
                right_y = []

                for i in range(n_samples):
                    if X[i][feature] <= threshold:
                        left_y.append(y[i])
                    else:
                        right_y.append(y[i])

                if len(left_y) == 0 or len(right_y) == 0:
                    continue

                left_pred = sum(left_y) / len(left_y)
                right_pred = sum(right_y) / len(right_y)

                error = 0
                for i in range(n_samples):
                    pred = left_pred if X[i][feature] <= threshold else right_pred
                    error += (y[i] - pred) ** 2

                if error < best_error:
                    best_error = error
                    self.feature_index = feature
                    self.threshold = threshold
                    self.left_value = left_pred
                    self.right_value = right_pred
    def predict(self, X):
        predictions = []
        for row in X:
            if row[self.feature_index] <= self.threshold:
                predictions.append(self.left_value)
            else:
                predictions.append(self.right_value)
        return predictions



In [20]:
class GradientBoostingRegressor:
    def __init__(self, n_estimators=10, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        self.init_prediction = 0
    def fit(self, X, y):
        self.init_prediction = sum(y) / len(y)
        y_pred = [self.init_prediction] * len(y)

        for _ in range(self.n_estimators):
            residuals = [y[i] - y_pred[i] for i in range(len(y))]

            stump = DecisionStump()
            stump.fit(X, residuals)

            predictions = stump.predict(X)
            y_pred = [
                y_pred[i] + self.learning_rate * predictions[i]
                for i in range(len(y))
            ]

            self.models.append(stump)
    def predict(self, X):
        y_pred = [self.init_prediction] * len(X)

        for stump in self.models:
            predictions = stump.predict(X)
            y_pred = [
                y_pred[i] + self.learning_rate * predictions[i]
                for i in range(len(X))
            ]

        return y_pred
    def sigmoid(x):
        return 1 / (1 + math.exp(-x))



In [21]:
class GradientBoostingClassifier:
    def __init__(self, n_estimators=10, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.models = []
        self.init_prediction = 0
    def fit(self, X, y):
        positive_ratio = sum(y) / len(y)
        self.init_prediction = math.log(positive_ratio / (1 - positive_ratio))

        F = [self.init_prediction] * len(y)

        for _ in range(self.n_estimators):
            probabilities = [sigmoid(F[i]) for i in range(len(y))]
            residuals = [y[i] - probabilities[i] for i in range(len(y))]

            stump = DecisionStump()
            stump.fit(X, residuals)

            updates = stump.predict(X)
            F = [
                F[i] + self.learning_rate * updates[i]
                for i in range(len(y))
            ]

            self.models.append(stump)
    def predict_proba(self, X):
        F = [self.init_prediction] * len(X)

        for stump in self.models:
            updates = stump.predict(X)
            F = [
                F[i] + self.learning_rate * updates[i]
                for i in range(len(X))
            ]

        return [sigmoid(f) for f in F]
    def predict(self, X):
        probabilities = self.predict_proba(X)
        return [1 if p >= 0.5 else 0 for p in probabilities]




In [22]:
print("=== Gradient Boosting Regression ===")

X_reg = [[1], [2], [3], [4], [5]]
y_reg = [1, 3, 3, 5, 5]

gbr = GradientBoostingRegressor(
    n_estimators=10,
    learning_rate=0.3
)

gbr.fit(X_reg, y_reg)
predictions = gbr.predict(X_reg)

print("Predictions:", predictions)
print("Actual:", y_reg)


=== Gradient Boosting Regression ===
Predictions: [1.20012570711, 3.0246493898475, 3.0246493898475, 4.8752877565974995, 4.8752877565974995]
Actual: [1, 3, 3, 5, 5]


In [24]:
import math

print("\n=== Gradient Boosting Classification ===")

X_clf = [[1], [2], [3], [4], [5], [6]]
y_clf = [0, 0, 0, 1, 1, 1]

gbc = GradientBoostingClassifier(
    n_estimators=10,
    learning_rate=0.3
)

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

print("Probabilities:", gbc.predict_proba(X_clf))
print("Predictions:", gbc.predict(X_clf))
print("Actual:", y_clf)



=== Gradient Boosting Classification ===
Probabilities: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
Predictions: [1, 1, 1, 1, 1, 1]
Actual: [0, 0, 0, 1, 1, 1]
